# 외부 데이터 증강

In [1]:
import re
import json
from pprint import pprint
from datasets import DatasetDict, Dataset, load_from_disk, load_metric, load_dataset
import pandas as pd

/opt/conda/envs/mrc/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets1 = load_from_disk("../data/train_dataset/")

datasets1_train = pd.DataFrame(datasets1['train'])
datasets1_valid = pd.DataFrame(datasets1['validation'])

train_datasets = [datasets1_train]
valid_datasets = [datasets1_valid]

In [3]:
print(datasets1)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [4]:
print(train_datasets[0].shape,valid_datasets[0].shape)

(3952, 7) (240, 7)


In [5]:
with open("../data/external_dataset/ko_wiki_v1_squad.json") as f:
    datasets2 = json.load(f)

def get_data_frame_ko_wiki(dataset):
    dataset = dataset['data']
    titles = []
    contexts = []
    questions = []
    answers = []
    ids = []
    # print(type(dataset))
    for data in dataset:
        titles.append(data['title'])
        contexts.append(data['paragraphs'][0]['context'])
        questions.append(data['paragraphs'][0]['qas'][0]['question'])
        answer = data['paragraphs'][0]['qas'][0]['answers'][0]
        answer['text'] = [answer['text']]
        answer['answer_start'] = [answer['answer_start']]
        answers.append(answer)
        ids.append(data['paragraphs'][0]['qas'][0]['id'])
    df = pd.DataFrame({'title': titles, 'context': contexts, 'question': questions, 'id': ids, 'answers': answers})
    return df

datasets2 = get_data_frame_ko_wiki(datasets2).drop_duplicates(['context', 'question'], ignore_index=True)
valid_start_idx = len(datasets2) - len(datasets2) // 10
datasets2_train = datasets2[:valid_start_idx]
datasets2_valid = datasets2[valid_start_idx:]
train_datasets.append(datasets2_train)
valid_datasets.append(datasets2_valid)

In [6]:
print(train_datasets[1].shape,valid_datasets[1].shape)

(61223, 5) (6802, 5)


In [7]:
with open("../data/external_dataset/KorQuAD_v1.0_train.json", "r") as f:
    datasets3_train = json.load(f)

with open("../data/external_dataset/KorQuAD_v1.0_dev.json", "r") as f:
    datasets3_valid = json.load(f)

def get_data_frame_korquad(dataset):
    dataset = dataset['data']
    titles = []
    contexts = []
    questions = []
    answers = []
    ids = []
    # print(type(dataset))
    for data in dataset:
        # print(type(data))
        for pg in data['paragraphs']:
            # print(pg)
            for qa in pg['qas']:
                titles.append(data['title'])
                contexts.append(pg['context'])
                questions.append(qa['question'])
                answer = qa['answers'][0]
                answer['text'] = [answer['text']]
                answer['answer_start'] = [answer['answer_start']]
                answers.append(answer)
                ids.append(qa['id'])
    df = pd.DataFrame({'title': titles, 'context': contexts, 'question': questions, 'id': ids, 'answers': answers})
    return df

datasets3_train = get_data_frame_korquad(datasets3_train)
datasets3_valid = get_data_frame_korquad(datasets3_valid)

train_datasets.append(datasets3_train)
valid_datasets.append(datasets3_valid)

In [8]:
print(train_datasets[2].shape,valid_datasets[2].shape)

(60407, 5) (5774, 5)


In [9]:
train_datasets[0]

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548
...,...,...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,"{'answer_start': [317], 'text': ['지질 구조']}",8275,3445
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,"{'answer_start': [616], 'text': ['뉴저지']}",37764,450
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,"{'answer_start': [255], 'text': ['칼 에케르트']}",12730,2252
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,"{'answer_start': [476], 'text': ['슬라브 신화']}",10924,2595


In [10]:
train_datasets[1]

,title,context,question,id,answers
0,다테_기미코,"재팬 오픈에서 4회 우승하였으며, 통산 단식 200승 이상을 거두었다. 1994년 ...",다테 기미코가 최초로 은퇴 선언을 한게 언제지,9_f2_wiki_2822-1,"{'answer_start': [260], 'text': ['1996년 9월 24일']}"
1,Ave;new,"ave;new(아베;뉴, アベニュー)는 도쿄 치요다구에 본 거처를 둔 일본의 음악 ...",ave;new 본거지 어디야,9_f2_wiki_3091-1,"{'answer_start': [22], 'text': ['도쿄 치요다구']}"
2,사카이_다다요시_(1714년),"사카이 다다요시(일본어: 酒井忠休, 1714년 9월 24일 ~ 1787년 6월 3일...",사카이 다다요시의 아버지가 누구지,9_f7_wiki_3128-1,"{'answer_start': [182], 'text': ['사카이 나오타카(酒井直..."
3,일반성면,"일반성면은 동부 5개 면의 교통, 문화, 교육, 상업의 중심지로서 일찍부터 상업이 ...",일반성면의 면적이 얼마야,9_f9_wiki_2703-1,"{'answer_start': [84], 'text': ['19.41 km²']}"
4,금나라,"지방은 전국을 19개 로(路)로 나누고, 그 아래에 부(府)나 주(州)를 두고, 다...",금나라를 세운 사람이 누구야,8_C59_wiki_1533-1,"{'answer_start': [247], 'text': ['태조 아구다']}"
...,...,...,...,...,...
61218,터키 요리,터키식 식사에서 가장 먼저 나오는 음식이 초르바(çorba)라고 불리는 수프이다. ...,초르바가 뭐야,7_c38_wiki_820-1,"{'answer_start': [0], 'text': ['터키식 식사에서 가장 먼저..."
61219,안재홍,김익렬이 증거로 제출한 사진첩을 들여다보던 윌리엄 F. 딘 장관은 흥분하여 사진 자...,조병옥이 공산주의자로 지목한 사람은 누구야,7_c45_wiki_110-1,"{'answer_start': [93], 'text': ['김익렬']}"
61220,두더지,큰두더지(또는 일본두더지 Mogera wogura)의 아종으로 간주하기도 한다. 몸...,두더지는 보통 무슨 색이야,8_C40_wiki_1089-1,"{'answer_start': [121], 'text': ['암갈색 또는 흑갈색']}"
61221,오스카르 카르도소,"2009년 8월 31일, 비토리아 FC과의 리그 3라운드 경기에서 해트트릭을 기록하...",포르투갈 리그를 뭐라고 불러,8_C38_wiki_861-1,"{'answer_start': [367], 'text': ['프리메이라리가']}"


In [11]:
train_datasets[2]

,title,context,question,id,answers
0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,6566495-0-0,"{'text': ['교향곡'], 'answer_start': [54]}"
1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,6566495-0-1,"{'text': ['1악장'], 'answer_start': [421]}"
2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,6566495-0-2,"{'text': ['베토벤의 교향곡 9번'], 'answer_start': [194]}"
3,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,6566518-0-0,"{'text': ['파우스트'], 'answer_start': [15]}"
4,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,6566518-0-1,"{'text': ['합창교향곡'], 'answer_start': [354]}"
...,...,...,...,...,...
60402,뉴델리_메탈로-베타락타마제,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,6467478-1-1,"{'text': ['인도'], 'answer_start': [73]}"
60403,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,6467478-2-0,"{'text': ['유전자 전달'], 'answer_start': [253]}"
60404,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,6467478-2-1,"{'text': ['37건'], 'answer_start': [129]}"
60405,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,6490801-2-0,"{'text': ['콜리스틴'], 'answer_start': [404]}"


In [13]:
train_dataset = pd.concat(train_datasets, ignore_index=True)
valid_dataset = pd.concat(valid_datasets, ignore_index=True)

In [14]:
train_dataset = train_dataset.drop_duplicates(['context', 'question'], ignore_index=True)
valid_dataset = valid_dataset.drop_duplicates(['context', 'question'], ignore_index=True)

train_dataset = Dataset.from_pandas(train_dataset)
valid_dataset = Dataset.from_pandas(valid_dataset)

In [16]:
datasets = DatasetDict({'train': train_dataset, 'validation': valid_dataset})

In [17]:
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 125435
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 12806
    })
})


In [19]:
datasets.save_to_disk("data_wiki_korquad")